In [ ]:
"""

Look at the distribution of the weights for different sigma_mod


"""

In [ ]:
import xarray as xr
import numpy as np
from tqdm.notebook import tqdm
import seaborn as sns
import multimelt.useful_functions as uf
import os
from xhistogram.xarray import histogram
import scipy.io


In [ ]:
sns.set_context('paper')

In [ ]:
%matplotlib qt5

READ IN DATA

In [ ]:
home_path = '/bettik/burgardc/'
plot_path = '/bettik/burgardc/PLOTS/summer_paper_plots/'
outputpath_GL = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/GL_FLUX/'
outputpath_weights = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/ANALYSIS/'
inputpath_raw = '/bettik/burgardc/DATA/SUMMER_PAPER/raw/'
inputpath_data=home_path+'/DATA/SUMMER_PAPER/interim/'


In [ ]:
inputpath_mask = home_path+'/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
file_isf_orig = xr.open_dataset(inputpath_mask+'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)

In [ ]:
# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

In [ ]:
BedMachine_orig = xr.open_dataset(inputpath_data+'BedMachine_v2_aggregated4km_allvars.nc')
file_BedMachine = uf.cut_domain_stereo(BedMachine_orig, map_lim, map_lim)
file_isf_conc = file_BedMachine['isf_conc']

grid_cell_area_file = xr.open_dataset(inputpath_data+'gridarea_ISMIP6_AIS_4000m_grid.nc').sel(x=file_isf.x,y=file_isf.y)
true_grid_cell_area = grid_cell_area_file['cell_area'].drop('lon').drop('lat')
cell_area_weight = true_grid_cell_area/(4000 * 4000)

lon = file_isf.longitude
lat = file_isf.latitude

xx = file_isf.x
yy = file_isf.y
dx = (xx[2] - xx[1]).values
dy = (yy[2] - yy[1]).values
grid_cell_area_const = abs(dx*dy)  
grid_cell_area_weighted = file_isf_conc * grid_cell_area_const * cell_area_weight

isf_stack_mask = uf.create_stacked_mask(file_isf['ISF_mask'], file_isf.Nisf, ['y','x'], 'mask_coord')


In [ ]:
area_list = []
for kisf in tqdm(file_isf.Nisf):
    true_grid_cell_area_kisf = uf.choose_isf(grid_cell_area_weighted ,isf_stack_mask, kisf) 
    area_list.append(true_grid_cell_area_kisf.sum().assign_coords({'Nisf': kisf}))
area_BedMachine = xr.concat(area_list, dim = 'Nisf')/10**6

In [ ]:
sorted_isf_rignot = [11,69,43,28,12,57,
                     70,44,29,13,58,71,45,30,14,
                     59,72,46,
                     31,
                     15,61,73,47,32,16,48,33,17,62,49,34,18,63,74,
                     50,35,19,64,
                     10,
                     36,20,65,51,37,
                     22,38,52,23,66,53,39,24,
                     67,40,54,75,25,41,
                     26,42,55,68,60,27]

REFERENCE

In [ ]:
inputpath_davison = home_path+'/DATA/SUMMER_PAPER/raw/'
davison_file0 = xr.open_dataset(inputpath_davison + 'steadystate_davison23.nc')
davison_file = xr.open_dataset(inputpath_davison + 'varying_conditions_davison23.nc')
greene_file = xr.open_dataset(inputpath_davison + 'area_isf_greene22.nc')

In [ ]:
mass_balance_davison_steady = davison_file0['basal_melt_obs'] - davison_file0['SMB_obs'] + davison_file0['calving_obs'] - davison_file0['discharge_obs']
mass_balance_davison_steady_unc = np.sqrt(davison_file0['basal_melt_unc']**2 + davison_file0['SMB_unc']**2 + davison_file0['calving_obs']**2 + davison_file0['discharge_unc']**2)

In [ ]:
mass_balance_davison_varying = davison_file['melt_abs'] - davison_file['SMB_abs'] + davison_file0['calving_obs'] - davison_file['discharge_abs']
mass_balance_davison_varying_unc = np.sqrt(davison_file['melt_unc']**2 + davison_file['SMB_unc']**2 + davison_file0['calving_unc']**2 + davison_file['discharge_unc']**2)

SMB

In [ ]:
inputpath_atmo_Chris = '/bettik/burgardc/DATA/SUMMER_PAPER/raw/TS_SMB_DATA/out2/'
inputpath_atmo_Nico = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/SMB_EMULATED/'
scenario = 'historical'

melt_atmo_list = []
for mod in ['CESM2','CNRM-CM6-1','IPSL-CM6A-LR','MPI-ESM1-2-HR','UKESM1-0-LL']:
    if os.path.exists(inputpath_atmo_Chris+mod+'_SMB_'+scenario+'-1980-2014.nc'):
        melt_atmo = xr.open_dataset(inputpath_atmo_Chris+mod+'_SMB_'+scenario+'-1980-2014.nc').sel(Nisf=rignot_isf)
        melt_atmo['time'] = melt_atmo['time'].dt.year
        melt_atmo_list.append(melt_atmo['SMB'].assign_coords({'model': mod}))
    else:
        melt_atmo = xr.open_dataset(inputpath_atmo_Nico+mod+'_SMB_'+scenario+'-1980-2014.nc').sel(Nisf=rignot_isf)
        melt_atmo_list.append(melt_atmo['SMB'].assign_coords({'model': mod}))
        
for mod in ['ACCESS-ESM1-5','ACCESS-CM2','CESM2-WACCM','CNRM-ESM2-1',
            'CanESM5','GFDL-CM4','GFDL-ESM4','GISS-E2-1-H', 
           'MRI-ESM2-0']:
    melt_atmo = xr.open_dataset(inputpath_atmo_Nico+mod+'_SMB_'+scenario+'-1980-2014.nc').sel(Nisf=rignot_isf)
    melt_atmo_list.append(melt_atmo['SMB'].assign_coords({'model': mod}))
    
melt_atmo_all = xr.concat(melt_atmo_list, dim='model')

In [ ]:
melt_atmo_all_corr = melt_atmo_all / area_BedMachine * greene_file['area_abs']

BASAL MELT

In [ ]:
param_classic_list = ['linear_local',
              'quadratic_local','quadratic_local_locslope',
              'lazero19',
              'boxes_4_pismyes_picopno']

param_NN_list = ['NNsmall']

In [ ]:
## Melt outputpath
Gt_allmod_list = []
box1_allmod_list = []

scenario = 'historical'
for mod in ['ACCESS-ESM1-5','ACCESS-CM2','CESM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
            'CanESM5','GFDL-CM4','GFDL-ESM4','GISS-E2-1-H', 'IPSL-CM6A-LR',
           'MPI-ESM1-2-HR','MRI-ESM2-0','UKESM1-0-LL']: #
    
    outputpath_melt = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/OCEAN_MELT_RATE_CMIP/'+mod+'/'

    melt1D_list = []
    for mparam in param_classic_list:
        melt1D_scenario = xr.open_dataset(outputpath_melt + 'eval_metrics_1D_'+mparam+'_'+scenario+'_oneFRIS_anoISMIP.nc')
        melt1D_list.append(melt1D_scenario.assign_coords({'param':mparam}))
    melt1D_classic = xr.concat(melt1D_list, dim='param')       
    Gt_classic = melt1D_classic['melt_1D_Gt_per_y'].sel(time=range(1980,2015))

    melt1D_list = []
    for mparam in param_NN_list:
        melt1D_scenario = xr.open_dataset(outputpath_melt + 'evalmetrics_1D_'+mparam+'_'+scenario+'_anoISMIP.nc')
        melt1D_list.append(melt1D_scenario.assign_coords({'param':mparam}))
    melt1D_NN = xr.concat(melt1D_list, dim='param')   
    Gt_NN = melt1D_NN['predicted_melt'].sel(metrics='Gt')

    Gt_all = xr.concat([Gt_classic, Gt_NN], dim='param')
    
    Gt_allmod_list.append(Gt_all.assign_coords({'model': mod}))

Gt_allmod = xr.concat(Gt_allmod_list, dim='model').sel(time=range(1980,2015))
    

In [ ]:
Gt_allmod_corr = Gt_allmod / area_BedMachine * greene_file['area_abs']

In [ ]:
## Melt outputpath
Gt_allmod_list = []
box1_allmod_list = []

scenario = 'historical'
for mod in ['ACCESS-ESM1-5','ACCESS-CM2','CESM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
            'CanESM5','GFDL-CM4','GFDL-ESM4','GISS-E2-1-H', 'IPSL-CM6A-LR',
           'MPI-ESM1-2-HR','MRI-ESM2-0','UKESM1-0-LL']: #
    
    outputpath_melt = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/OCEAN_MELT_RATE_CMIP/'+mod+'/'

    melt1D_list = []
    for mparam in param_classic_list:
        melt1D_scenario = xr.open_dataset(outputpath_melt + 'eval_metrics_1D_'+mparam+'_'+scenario+'_oneFRIS_anoNEMO.nc')
        melt1D_list.append(melt1D_scenario.assign_coords({'param':mparam}))
    melt1D_classic = xr.concat(melt1D_list, dim='param')       
    Gt_classic = melt1D_classic['melt_1D_Gt_per_y'].sel(time=range(1980,2015))

    melt1D_list = []
    for mparam in param_NN_list:
        melt1D_scenario = xr.open_dataset(outputpath_melt + 'evalmetrics_1D_'+mparam+'_'+scenario+'_anoNEMO.nc')
        melt1D_list.append(melt1D_scenario.assign_coords({'param':mparam}))
    melt1D_NN = xr.concat(melt1D_list, dim='param')   
    Gt_NN = melt1D_NN['predicted_melt'].sel(metrics='Gt')

    Gt_all = xr.concat([Gt_classic, Gt_NN], dim='param')
    
    Gt_allmod_list.append(Gt_all.assign_coords({'model': mod}))

Gt_allmod_anoNEMO = xr.concat(Gt_allmod_list, dim='model').sel(time=range(1980,2015))
    

In [ ]:
Gt_allmod_anoNEMO_corr = Gt_allmod_anoNEMO / area_BedMachine * greene_file['area_abs']

GL FLUX ABUMIP

In [ ]:
#GL_flux_ABUMIP = xr.open_dataset(outputpath_GL + 'all_GL_fluxes.nc')
GL_flux = xr.open_dataset(outputpath_GL + 'all_GL_fluxes_varying_m.nc')

In [ ]:
GL_flux_ag = xr.Dataset()
GL_flux_ag['flux_Gt_ref'] = xr.concat([GL_flux['flux_Gt_ref_m1'].assign_coords({'m': 1}),
                                       GL_flux['flux_Gt_ref_m3'].assign_coords({'m': 3}),
                                       GL_flux['flux_Gt_ref_m5'].assign_coords({'m': 5})], dim='m')
GL_flux_ag['flux_Gt_ABUMIP'] = xr.concat([GL_flux['flux_Gt_ABUMIP_m1'].assign_coords({'m': 1}),
                                       GL_flux['flux_Gt_ABUMIP_m3'].assign_coords({'m': 3}),
                                       GL_flux['flux_Gt_ABUMIP_m5'].assign_coords({'m': 5})], dim='m')

SUM OF ALL VARIABLES

In [ ]:
mass_balance_simu_anoISMIP = Gt_allmod_corr - melt_atmo_all_corr + davison_file0['calving_obs'] - GL_flux_ag['flux_Gt_ref']
mass_balance_simu_anoNEMO = Gt_allmod_anoNEMO_corr - melt_atmo_all_corr + davison_file0['calving_obs'] - GL_flux_ag['flux_Gt_ref']

SENSITIVITY EXPERIMENTS AND LOOK AT DISTRIBUTIONS

In [ ]:
ano_choice = xr.open_dataset(outputpath_weights + 'NEMOorISMIP_choice_and_weights.nc')['ano_choice']

In [ ]:
mass_balance_simu_combined_list = []

for kisf in file_isf.Nisf:
    if ano_choice.sel(Nisf=kisf).values == 'NEMO':
        mass_balance_simu_combined_list.append(mass_balance_simu_anoNEMO.sel(Nisf=kisf))
        
    elif ano_choice.sel(Nisf=kisf).values == 'ISMIP':
        mass_balance_simu_combined_list.append(mass_balance_simu_anoISMIP.sel(Nisf=kisf))

mass_balance_simu_combined = xr.concat(mass_balance_simu_combined_list, dim='Nisf')

In [ ]:
mult_fac = 1

In [ ]:
diff_mod_obs = mass_balance_simu_combined.mean('time') - mass_balance_davison_varying.mean('time') 

sigma_obs = mass_balance_davison_varying_unc.mean('time')
sigma_mod = mass_balance_simu_combined.std('time').mean(['model','param','m']) * mult_fac

s_j = np.exp(-((diff_mod_obs)**2/(sigma_obs**2 + sigma_mod**2))) 

weight = (s_j / (s_j.sum(['model','param','m'])))

In [ ]:

f = plt.figure()

for i,ddim in enumerate(['param','model','m']): #

    if ddim == 'param':
        weight_ddim = weight.stack(stack_dim=['model','m','Nisf'])
    elif ddim == 'model':
        weight_ddim = weight.stack(stack_dim=['param','m','Nisf'])
    elif ddim == 'm':        
        weight_ddim = weight.stack(stack_dim=['param','model','Nisf'])


    bins = weight_ddim[ddim]
    _, bin_edges = np.histogram(weight_ddim.values, bins=np.arange(len(bins)), weights=weight_ddim.values)

    hist = weight_ddim.sum('stack_dim') # / weight_ddim.sum()

    ax[i] = f.add_subplot(1,3,i+1)
    ax[i].barh(weight_ddim[ddim], hist,  edgecolor='black') #width=1,
    ax[i].set_title('sigma_mod = '+str(mult_fac)+' x std deviation')


In [ ]:
for kisf in sorted_isf_rignot:
        f = plt.figure()

        for i,ddim in enumerate(['param','model','m']): #

            if ddim == 'param':
                weight_ddim = weight.sel(Nisf=kisf).stack(stack_dim=['model','m'])
            elif ddim == 'model':
                weight_ddim = weight.sel(Nisf=kisf).stack(stack_dim=['param','m'])
            elif ddim == 'm':        
                weight_ddim = weight.sel(Nisf=kisf).stack(stack_dim=['param','model'])

    
            bins = weight_ddim[ddim]
            _, bin_edges = np.histogram(weight_ddim.values, bins=np.arange(len(bins)), weights=weight_ddim.values)

            hist = weight_ddim.sum('stack_dim') # / weight_ddim.sum()
            
            ax[i] = f.add_subplot(1,3,i+1)
            ax[i].barh(weight_ddim[ddim], hist,  edgecolor='black') #width=1,
            ax[i].set_title('sigma_mod = '+str(mult_fac)+' x std deviation')
        f.suptitle(file_isf['isf_name'].sel(Nisf=kisf).values)

In [ ]:
plt.close('all')

In [ ]:

### THINK ABOUT THE SIGMA_MOD AND SIGMA_OBS AGAIN

regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','lightseagreen',
          'yellowgreen','royalblue','tomato','darkslateblue','purple','darkgoldenrod']

f = plt.figure()
f.set_size_inches(8.25*3, 8.25*3)

ax={}

leg_hdl = []

i = 0

sigma_tot_1 = np.sqrt(sigma_obs**2 + sigma_mod**2)
#sigma_tot_2 = np.sqrt(sigma_obs**2 + sigma_mod**2)
#sigma_tot_5 = np.sqrt(sigma_obs**2 + sigma_mod**2)
#sigma_tot_10 = np.sqrt(sigma_obs**2 + sigma_mod**2)



for kisf in tqdm(sorted_isf_rignot):
    
    ax[i] = f.add_subplot(8,8,i+1)
    
    melt_obs_kisf = mass_balance_davison_varying.sel(Nisf=kisf).sel(time=range(1997,2015)).mean('time')
    melt_obs_unc_kisf = mass_balance_davison_varying_unc.sel(Nisf=kisf).sel(time=range(1997,2015)).mean('time')
    ax[i].axhline(y=melt_obs_kisf,color='k')
    ax[i].fill_between(x=range(1997,2015), y1=melt_obs_kisf-melt_obs_unc_kisf, y2=melt_obs_kisf+melt_obs_unc_kisf, color='lightgrey')
    #ax[i].fill_between(x=range(1997,2015), y1=melt_obs_kisf-sigma_tot_10.sel(Nisf=kisf), y2=melt_obs_kisf+sigma_tot_10.sel(Nisf=kisf), color='lightgrey',alpha=0.2)
    #ax[i].fill_between(x=range(1997,2015), y1=melt_obs_kisf-sigma_tot_5.sel(Nisf=kisf), y2=melt_obs_kisf+sigma_tot_5.sel(Nisf=kisf), color='lightgrey',alpha=0.4)
    #ax[i].fill_between(x=range(1997,2015), y1=melt_obs_kisf-sigma_tot_2.sel(Nisf=kisf), y2=melt_obs_kisf+sigma_tot_2.sel(Nisf=kisf), color='lightgrey',alpha=0.6)
    ax[i].fill_between(x=range(1997,2015), y1=melt_obs_kisf-sigma_tot_1.sel(Nisf=kisf), y2=melt_obs_kisf+sigma_tot_1.sel(Nisf=kisf), color='lightgrey',alpha=0.8)




    for mmodel in mass_balance_simu_combined.model:
        for m in mass_balance_simu_combined.m:
            for mparam in mass_balance_simu_combined.param:
                ax[i].plot(mass_balance_simu_combined.time, mass_balance_simu_combined.sel(param=mparam,Nisf=kisf,m=m, model=mmodel), color='deepskyblue',alpha=0.05)

    #ax[i].plot(ensstat_weighted.time, ensstat_nonweighted.sel(quantile=0.5,Nisf=kisf), color='deepskyblue')
    #ax[i].fill_between(x=ensstat_weighted_combined.time, y1=ensstat_weighted_combined.sel(quantile=0.33,Nisf=kisf), y2=ensstat_weighted_combined.sel(quantile=0.66,Nisf=kisf), color='deepskyblue',alpha=0.2)

    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf['isf_name'].sel(Nisf=kisf).values))

    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1

#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
f.savefig(plot_path + 'checking_sigma_mod.pdf')

In [ ]:
for ddim in ['m']: #'param','model',
    
    if ddim == 'param':
        weight_ddim = weight.stack(stack_dim=['model','m','Nisf'])
    elif ddim == 'model':
        weight_ddim = weight.stack(stack_dim=['param','m','Nisf'])
    elif ddim == 'm':        
        weight_ddim = weight.stack(stack_dim=['param','model','Nisf'])
        
    xr_ddim, _ = xr.broadcast(weight_ddim[ddim], weight_ddim)

    arr_range = xr_ddim.copy()
    for nn, mdim in enumerate(xr_ddim[ddim].values):
        arr_range = arr_range.where(xr_ddim != mdim, nn)
    
    # Define bins
    bins = np.arange(len(weight_ddim[ddim]))

    # Compute histogram
    hist, bin_edges = np.histogram(arr_range.values, bins=np.arange(len(bins)), weights=weight_ddim.values)
    
    plt.figure()
    # Normalize histogram by weights
    hist = hist / weight_ddim.sum().values

    # Plot the weighted histogram
    plt.bar(bin_edges[:-1], hist, width=np.diff(bin_edges), edgecolor='black')
    
    plt.title(ddim)

In [ ]:
arr_range.values

In [ ]:
weight_ddim.sel(m=3).sum('stack_dim')

In [ ]:
weight_param.sum().values

In [ ]:
LHS = scipy.io.loadmat('/bettik/burgardc/DATA/SUMMER_PAPER/raw/LHSensemble.mat')

In [ ]:
scipy.io.whosmat('/bettik/burgardc/DATA/SUMMER_PAPER/raw/LHSensemble.mat')

In [ ]:
data = LHS['LHS']

In [ ]:
plt.figure()
histogram(xr_param.rename('test'), weights=weight_param) #, weights=weight , bins=weight.param.values.tolist()

In [ ]:
weight_param

In [ ]:
weight_param.rename('test')

In [ ]:
N, edges = np.histogram(data, bins=len(weight_param.param))

In [ ]:
def weighted_histogram(data, w, bin_number):
    N, edges = np.histogram(data, bins=bin_number)  # histogram without weight for edges and normalization
    bin_center = (edges[:-1] + edges[1:]) / 2

    N_noweight = N  # save N without weight -- used for normalization
    N = np.zeros_like(N)

    # calculate N per bin
    for i in range(len(data)):
        for b in range(len(edges) - 1):
            if edges[b] <= data[i] <= edges[b + 1]:
                N[b] += w[i]

    N /= N_noweight  # normalization for pdf

    return bin_center, N